In [13]:
import pymysql

# 连接数据库
conn1 = pymysql.connect(host='172.28.72.225', user='root', port=7487,
                       password='l84kCG5KP4uNNtRX', database='lynntest')

In [14]:
from sshtunnel import SSHTunnelForwarder
server = SSHTunnelForwarder(
    ('172.28.72.225', 22), # 指定 ssh 登录的跳转机的 address，端口号
    ssh_username='root', # 远程服务器的用户名，注意不是 DB 的用户名和密码
    ssh_password='GDS&123softswitch', # 远程服务器的密码
    remote_bind_address=('172.28.160.99', 13306), # 注意端口号不要加引号
    # local_bind_address=('127.0.0.1', 3306)#端口可以自己设置，没有冲突即可，不加这个参数也可
)
server.start()

In [20]:
# 数据库表是com_tran_info userlog 的sada_gdpi_click
conn2 = pymysql.connect(host='127.0.0.1',
                port=server.local_bind_port, # server.local_bind_port, 
                user='duser',
                password='RAkj#7853',
                database='userlog',
                connect_timeout=10,
            )

In [21]:
conn2

In [202]:
'''
pip install pymysql
pip install sshtunnel
'''
 
import pymysql
import pandas as pd
from sshtunnel import SSHTunnelForwarder
 
source_table = 'sada_gdpi_click_dtl'
target_table = 'example_table'

def get_all_data(cursor, table_name):
    cursor.execute("select * from " + table_name)
    results = cursor.fetchall()
    description = cursor.description
    # table head
    df = pd.DataFrame(data=results, columns=[item[0] for item in description])
    return df
 
 
df = get_all_data(conn2.cursor(), table_name=source_table)
# 保存 excel
# df.to_excel('./cold_source_run_data_history.xls',
#     sheet_name='cold_source_run_data_history',
#     index=False)


In [203]:
cur1 = conn1.cursor()
cur2 = conn2.cursor()

In [184]:
# cur2.execute("select * from " + source_table)
# results = cur2.fetchall()
# description = cursor.description
# # table head
# df = pd.DataFrame(data=results, columns=[item[0] for item in description])
# len(results)


In [204]:
# 删除表
create_query = 'DROP TABLE IF EXISTS %s'% (target_table)
cur1.execute(create_query)

0

In [209]:
df.keys()

Index(['sid', 'f_srcip', 'f_ad', 'f_ts', 'f_url', 'f_ref', 'f_ua', 'f_dstip',
       'f_cookie', 'f_src_port', 'f_json', 'f_update_time', 'f_dataid'],
      dtype='object')

In [210]:
# 执行查询语句
cur2.execute(f"SHOW COLUMNS FROM {source_table}")

# 获取查询结果
results = cur2.fetchall()
results

(('sid', 'bigint(20)', 'NO', 'PRI', None, 'auto_increment'),
 ('f_srcip', 'text', 'YES', '', None, ''),
 ('f_ad', 'text', 'YES', '', None, ''),
 ('f_ts', 'bigint(20)', 'YES', '', None, ''),
 ('f_url', 'text', 'YES', '', None, ''),
 ('f_ref', 'text', 'YES', '', None, ''),
 ('f_ua', 'text', 'YES', '', None, ''),
 ('f_dstip', 'text', 'YES', '', None, ''),
 ('f_cookie', 'text', 'YES', '', None, ''),
 ('f_src_port', 'text', 'YES', '', None, ''),
 ('f_json', 'text', 'YES', '', None, ''),
 ('f_update_time', 'datetime', 'YES', '', None, ''),
 ('f_dataid', 'bigint(20)', 'YES', '', None, ''))

In [211]:
create_query = 'CREATE TABLE IF NOT EXISTS %s ('%target_table
for k in results:
    create_query += '%s %s, '%(k[0],k[1])
create_query = create_query[:-2]
create_query += ')'
create_query
# cur1.execute(create_query)

'CREATE TABLE IF NOT EXISTS example_table (sid bigint(20), f_srcip text, f_ad text, f_ts bigint(20), f_url text, f_ref text, f_ua text, f_dstip text, f_cookie text, f_src_port text, f_json text, f_update_time datetime, f_dataid bigint(20))'

In [212]:
cur1.execute(create_query)

0

In [213]:
# 显示所有表名
cur1.execute('SHOW TABLES')
tables = cur1.fetchall()
tables

(('example_table',), ('oriTableRelation',), ('oriTablesT1',), ('oriTablesT2',))

In [215]:
insert_data = df.values.tolist()

In [216]:
insert_query = 'INSERT INTO example_table ('
for k in df.keys():
    insert_query += '%s, '%k
insert_query = insert_query[:-2] + ') VALUES ('
for i in range(len(df.keys())):
    insert_query += '%s, '
insert_query = insert_query[:-2]+')'
insert_query


'INSERT INTO example_table (sid, f_srcip, f_ad, f_ts, f_url, f_ref, f_ua, f_dstip, f_cookie, f_src_port, f_json, f_update_time, f_dataid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [179]:
# insert_query = "INSERT INTO example_table (srcip, ad) VALUES (%s, %s)"
# insert_data = df[:2].values[:,:2].tolist()
# insert_data[0]

In [217]:
cur1.executemany(insert_query, insert_data)
# cur1.close()

130000

In [218]:
conn1.commit()

### 拆分存储

In [1]:
import yaml
yaml_file = 'dataplatform_config-k=2-n=6-1726885230.yaml'

In [20]:
fs = open(yaml_file, encoding="UTF-8")
datas = yaml.load(fs, Loader=yaml.FullLoader)

In [23]:
import json
 
# 假设有一个名为data.json的文件
json_file_path = 'table_config.json'
 
# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as file:
    datas = json.load(file)
 
# 现在data包含了JSON文件中的数据，可以按需使用
print(data)

{'table-config': {'tables': [{'real_table_name': 'dataplatform_config'}, {'relation_table_name': 'dataplatform_config_relation'}, {'split_tables': [{'split_table_name': 'server0', 'columns': [{'name': 'srcip', 'type': 'VARCHAR', 'length': 100}]}, {'split_table_name': 'server1', 'columns': [{'name': 'ua', 'type': 'VARCHAR', 'length': 100}, {'name': 'dstip', 'type': 'VARCHAR', 'length': 100}, {'name': 'cookie', 'type': 'VARCHAR', 'length': 100}]}, {'split_table_name': 'server2', 'columns': [{'name': 'ref', 'type': 'VARCHAR', 'length': 100}, {'name': 'url', 'type': 'VARCHAR', 'length': 100}, {'name': 'ts', 'type': 'VARCHAR', 'length': 100}, {'name': 'ad', 'type': 'VARCHAR', 'length': 100}]}, {'split_table_name': 'server3', 'columns': [{'name': 'dataid', 'type': 'VARCHAR', 'length': 100}, {'name': 'json', 'type': 'VARCHAR', 'length': 100}, {'name': 'update_time', 'type': 'VARCHAR', 'length': 100}, {'name': 'src_port', 'type': 'VARCHAR', 'length': 100}]}]}]}}


In [33]:
datas['table-config']['tables'][0]['real_table_name']

'dataplatform_config'

In [34]:
datas['table-config']['tables'][2]['split_tables']

[{'split_table_name': 'server0',
  'columns': [{'name': 'srcip', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server1',
  'columns': [{'name': 'ua', 'type': 'VARCHAR', 'length': 100},
   {'name': 'dstip', 'type': 'VARCHAR', 'length': 100},
   {'name': 'cookie', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server2',
  'columns': [{'name': 'ref', 'type': 'VARCHAR', 'length': 100},
   {'name': 'url', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ts', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ad', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server3',
  'columns': [{'name': 'dataid', 'type': 'VARCHAR', 'length': 100},
   {'name': 'json', 'type': 'VARCHAR', 'length': 100},
   {'name': 'update_time', 'type': 'VARCHAR', 'length': 100},
   {'name': 'src_port', 'type': 'VARCHAR', 'length': 100}]}]

In [29]:
datas['table-config']['tables'][1]['split_tables']

KeyError: 'split_tables'

In [12]:
split_info = datas['table-config']['tables'][2]['split_tables']
split_info

[{'split_table_name': 'server0',
  'columns': [{'name': 'srcip', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server1',
  'columns': [{'name': 'ua', 'type': 'VARCHAR', 'length': 100},
   {'name': 'dstip', 'type': 'VARCHAR', 'length': 100},
   {'name': 'cookie', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server2',
  'columns': [{'name': 'ref', 'type': 'VARCHAR', 'length': 100},
   {'name': 'url', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ts', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ad', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server3',
  'columns': [{'name': 'dataid', 'type': 'VARCHAR', 'length': 100},
   {'name': 'json', 'type': 'VARCHAR', 'length': 100},
   {'name': 'update_time', 'type': 'VARCHAR', 'length': 100},
   {'name': 'src_port', 'type': 'VARCHAR', 'length': 100}]}]

In [222]:
split_info[0]['split_table_name']

'server0'

In [225]:
split_info[0]['columns']

[{'name': 'srcip', 'type': 'VARCHAR', 'length': 100}]

In [231]:
split_info

[{'split_table_name': 'server0',
  'columns': [{'name': 'srcip', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server1',
  'columns': [{'name': 'ua', 'type': 'VARCHAR', 'length': 100},
   {'name': 'dstip', 'type': 'VARCHAR', 'length': 100},
   {'name': 'cookie', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server2',
  'columns': [{'name': 'ref', 'type': 'VARCHAR', 'length': 100},
   {'name': 'url', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ts', 'type': 'VARCHAR', 'length': 100},
   {'name': 'ad', 'type': 'VARCHAR', 'length': 100}]},
 {'split_table_name': 'server3',
  'columns': [{'name': 'dataid', 'type': 'VARCHAR', 'length': 100},
   {'name': 'json', 'type': 'VARCHAR', 'length': 100},
   {'name': 'update_time', 'type': 'VARCHAR', 'length': 100},
   {'name': 'src_port', 'type': 'VARCHAR', 'length': 100}]}]

In [278]:
for i in range(len(split_info)):
    table_name = 'p_'+split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    drop_query = 'DROP TABLE IF EXISTS %s'%table_name
    cur1.execute(drop_query)
    create_query = 'CREATE TABLE IF NOT EXISTS %s (sid INT, '%table_name
    for k in columns_info:
        create_query += '%s %s(%s), '%(k['name'], k['type'], k['length'])
    create_query = create_query[:-2]
    create_query += ')'
    create_query
    print(create_query)
    cur1.execute(create_query)

CREATE TABLE IF NOT EXISTS p_server0 (sid INT, srcip VARCHAR(100))
CREATE TABLE IF NOT EXISTS p_server1 (sid INT, ua VARCHAR(100), dstip VARCHAR(100), cookie VARCHAR(100))
CREATE TABLE IF NOT EXISTS p_server2 (sid INT, ref VARCHAR(100), url VARCHAR(100), ts VARCHAR(100), ad VARCHAR(100))
CREATE TABLE IF NOT EXISTS p_server3 (sid INT, dataid VARCHAR(100), json VARCHAR(100), update_time VARCHAR(100), src_port VARCHAR(100))


In [276]:
create_query

'CREATE TABLE IF NOT EXISTS p_server3 (sid, dataid VARCHAR(100), json VARCHAR(100), update_time VARCHAR(100), src_port VARCHAR(100))'

In [237]:
source_columns = df.keys().tolist()
source_columns

['sid',
 'f_srcip',
 'f_ad',
 'f_ts',
 'f_url',
 'f_ref',
 'f_ua',
 'f_dstip',
 'f_cookie',
 'f_src_port',
 'f_json',
 'f_update_time',
 'f_dataid']

In [298]:
target_columns = {}
for i in range(len(split_info)):
    table_name = 'p_'+split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    cross_columns = ['sid']
    for k in columns_info:
        match_name = 'f_'+k['name']
        if match_name in source_columns:
            cross_columns.append(k['name'])
    target_columns[table_name] = cross_columns
target_columns

{'p_server0': ['sid', 'srcip'],
 'p_server1': ['sid', 'ua', 'dstip', 'cookie'],
 'p_server2': ['sid', 'ref', 'url', 'ts', 'ad'],
 'p_server3': ['sid', 'dataid', 'json', 'update_time', 'src_port']}

In [252]:
target_columns.keys()

dict_keys(['p_server0', 'p_server1', 'p_server2', 'p_server3'])

In [299]:
# 插入子表
insert_query_dict = {}
for table_name in target_columns:
    insert_query = 'INSERT INTO %s ('%table_name
    columns_list = target_columns[table_name]
    for k in columns_list:
        insert_query += '%s, '%k
    insert_query = insert_query[:-2] + ') VALUES ('
    for i in range(len(columns_list)):
        insert_query += '%s, '
    insert_query = insert_query[:-2]+')'
    insert_query_dict[table_name] = insert_query
insert_query_dict

{'p_server0': 'INSERT INTO p_server0 (sid, srcip) VALUES (%s, %s)',
 'p_server1': 'INSERT INTO p_server1 (sid, ua, dstip, cookie) VALUES (%s, %s, %s, %s)',
 'p_server2': 'INSERT INTO p_server2 (sid, ref, url, ts, ad) VALUES (%s, %s, %s, %s, %s)',
 'p_server3': 'INSERT INTO p_server3 (sid, dataid, json, update_time, src_port) VALUES (%s, %s, %s, %s, %s)'}

In [302]:
target_columns[table_name]

['sid', 'srcip']

In [305]:
for table_name in target_columns:
    source_keys = ['sid']
    for k in target_columns[table_name]:
        if k !='sid':
            source_keys.append('f_'+k)
    insert_data = df[source_keys].values.tolist()
    insert_query = insert_query_dict[table_name]
    cur1.executemany(insert_query, insert_data)
    conn1.commit()
    # break
insert_query, insert_data

('INSERT INTO p_server3 (sid, dataid, json, update_time, src_port) VALUES (%s, %s, %s, %s, %s)',
 [[1, 6780877879247444511, '', Timestamp('2024-08-19 00:02:40'), 'A9D8'],
  [2, 4672126372713221517, '', Timestamp('2024-08-19 00:02:40'), 'A9D8'],
  [3, -5627499052269109921, '', Timestamp('2024-08-19 00:02:40'), 'A9D8'],
  [4, 8191034885009885891, '', Timestamp('2024-08-19 00:02:40'), 'CF56'],
  [5, 4779104489481930501, '', Timestamp('2024-08-19 00:02:40'), 'E984'],
  [6, 4945183873037052020, '', Timestamp('2024-08-19 00:02:40'), 'E377'],
  [7, -9169890660588059291, '', Timestamp('2024-08-19 00:02:40'), '83A0'],
  [8, -6375576059816258129, '', Timestamp('2024-08-19 00:02:40'), 'C5A9'],
  [9, -3101928439554651648, '', Timestamp('2024-08-19 00:02:40'), 'E85C'],
  [10, 2458913498372245562, '', Timestamp('2024-08-19 00:02:40'), '2DB7'],
  [11, -8372180975519134289, '', Timestamp('2024-08-19 00:02:40'), 'C480'],
  [12, 4249748022976549090, '', Timestamp('2024-08-19 00:02:40'), '368F'],
  [13, 